# Perocube Logbook Data Upload Notebook

This notebook processes and uploads logbook data from the Perocube Excel file to the TimescaleDB database.

## Purpose
- Read logbook data from 'PeroCube_logbook_example.xlsx'
- Parse and validate the data
- Upload the data to the TimescaleDB database
- Avoid duplicate data entries

## Prerequisites
- Running TimescaleDB instance (configured in docker-compose.yml)
- Access to the Perocube logbook Excel file
- Environment variables configured in .env file (for database connection)

## 1. Setup and Imports

Import required libraries and install any missing dependencies.

In [ ]:
# Core data processing libraries
import os
import pandas as pd
from datetime import datetime, timezone
from pathlib import Path

# Database libraries
from sqlalchemy import create_engine, text

# Progress tracking
from tqdm.notebook import tqdm

# Environment variables
from dotenv import load_dotenv

# Logging
import logging
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# Install required packages if not already installed
!pip install psycopg2-binary sqlalchemy pandas tqdm pathlib python-dotenv openpyxl
import psycopg2

## 2. Configuration

Load configuration from environment variables and set up constants.

In [ ]:
# Look for the .env file two directories up from the notebook location
dotenv_path = Path("../../.env")
load_dotenv(dotenv_path)

# Database configuration from environment variables with fallbacks
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': int(os.getenv('DB_PORT', 5432)),
    'database': os.getenv('DB_NAME', 'perocube'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'postgres')
}

# Print database connection info (excluding password)
print(f"Database connection: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']} as {DB_CONFIG['user']}")

# Data directory and file configuration
ROOT_DIRECTORY = os.getenv('DEFAULT_DATA_DIR', "../../sample_data/datasets/PeroCube-sample-data")
LOGBOOK_FILE = "PeroCube_logbook_example.xlsx"
LOGBOOK_SHEET = "Perocube history"

# Batch size for database operations
BATCH_SIZE = 1000

## 3. Read and Process Logbook Data

In [ ]:
# Create the full path to the logbook file
logbook_path = Path(ROOT_DIRECTORY) / LOGBOOK_FILE

# Read the Excel sheet, skip first row and use second row as header
try:
    df = pd.read_excel(logbook_path, sheet_name=LOGBOOK_SHEET, header=1)
    print(f"Successfully read {len(df)} rows from {LOGBOOK_FILE}")
    
    # Display the first few rows and data info
    print("\nColumn names:")
    print(df.columns.tolist())
    
    print("\nFirst few rows of the data:")
    display(df.head())
    
    print("\nDataset information:")
    display(df.info())
    
except Exception as e:
    print(f"Error reading Excel file: {str(e)}")

In [ ]:
# Analyze current dataframe state
print("Checking for unnamed columns:")
unnamed_cols = [col for col in df.columns if 'Unnamed' in str(col)]
print(f"Unnamed columns found: {unnamed_cols}")

print("\nCurrent data types:")
print(df.dtypes)

print("\nMissing values per column:")
print(df.isnull().sum())

print("\nTotal rows with all missing values:")
print(df.isna().all(axis=1).sum())

In [ ]:
# Clean the dataframe

# 1. Remove unnamed columns
df = df.loc[:, ~df.columns.str.contains('Unnamed')]

# 2. Drop rows where all values are missing
df = df.dropna(how='all')

# Print cleaning results
print("Dataframe shape after cleaning:")
print(f"Initial shape: {df.shape}")

# Display updated column list
print("\nUpdated column names:")
print(df.columns.tolist())

# Display updated missing values count
print("\nMissing values per column after cleaning:")
print(df.isnull().sum())

# Display first few rows of cleaned data
print("\nFirst few rows of cleaned data:")
display(df.head())

In [ ]:
# Remove completely empty Comment 2 column
df = df.drop('Comment 2', axis=1)

# Print updated dataframe info
print("Dataframe shape after removing Comment 2:")
print(f"Shape: {df.shape}")

# Display updated column list
print("\nUpdated column names:")
print(df.columns.tolist())

# Display first few rows of cleaned data
print("\nFirst few rows of cleaned data:")
display(df.head())

In [ ]:
# Fix data types

# 1. Convert date columns to datetime
df['Date removed'] = pd.to_datetime(df['Date removed'], format='%d.%m.%Y', errors='coerce')
df['Date installed'] = pd.to_datetime(df['Date installed'], format='%d.%m.%Y', errors='coerce')

# 2. Convert numeric columns
numeric_columns = ['Board', 'Channel', 'Status', 'Area', 'Init.PCE']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 3. Ensure string columns are properly formatted (strip whitespace)
string_columns = ['Cell type', 'Cell name', 'Pixel', 'Encap.', 'Structure', 
                  'Producer', 'Owner', 'Project', 'Temp sensor', 'Comment 1']
for col in string_columns:
    df[col] = df[col].astype(str).str.strip()
    # Replace 'nan' strings with actual NaN
    df[col] = df[col].replace('nan', pd.NA)

# Display the updated data types
print("Updated data types:")
print(df.dtypes)

# Display sample of the data to verify conversions
print("\nSample of converted data:")
display(df.head(25))

# Check for any conversion issues (invalid dates or numbers)
print("\nCount of NaN values after type conversion:")
print(df.isna().sum())

In [ ]:
# Remove rows where both dates are null
original_rows = len(df)
df = df.dropna(subset=['Date removed', 'Date installed'], how='all')

# Print results
print(f"Removed {original_rows - len(df)} rows where both dates were missing")
print(f"Remaining rows: {len(df)}")

# Display missing values count after removal
print("\nMissing values per column after removing rows with missing dates:")
print(df.isna().sum())

# Display sample of remaining data
print("\nSample of cleaned data:")
display(df.head(25))

In [ ]:
# Show rows where installation date is missing
missing_install_date = df[df['Date installed'].isna()]

print(f"Found {len(missing_install_date)} rows with missing installation date:\n")
display(missing_install_date)

In [ ]:
# Fill missing installation dates with January 1st, 2020
default_install_date = pd.to_datetime('2020-01-01')
df['Date installed'] = df['Date installed'].fillna(default_install_date)

# Verify the changes
print("Checking for any remaining missing installation dates:")
print(f"Missing installation dates: {df['Date installed'].isna().sum()}")

# Display the rows that were updated
print("\nVerifying the rows that were updated:")
display(df[df['Date installed'] == default_install_date])

In [ ]:
# Convert Board and Channel to integers (as per database schema)
df['Board'] = df['Board'].astype('Int64')  # Using Int64 to handle NaN values
df['Channel'] = df['Channel'].astype('Int64')  # Using Int64 to handle NaN values

# Verify the conversion
print("Updated data types for Board and Channel:")
print(df[['Board', 'Channel']].dtypes)

# Display a sample to verify the conversion
print("\nSample of Board and Channel data:")
display(df[['Board', 'Channel']].head(10))

In [ ]:
# Align numeric types with database schema
df['Area'] = df['Area'].astype('float64')
df['Init.PCE'] = df['Init.PCE'].astype('float64')

# Validate and truncate string columns to match VARCHAR(255)
string_columns = ['Cell type', 'Cell name', 'Pixel', 'Encap.', 'Structure', 
                 'Producer', 'Owner', 'Project', 'Temp sensor', 'Comment 1']

for col in string_columns:
    # Check if any string is longer than 255 characters
    mask = df[col].str.len() > 255
    if mask.any():
        print(f"Warning: Found {mask.sum()} values in {col} longer than 255 characters. Truncating...")
        df.loc[mask, col] = df.loc[mask, col].str.slice(0, 255)

# Display updated data types
print("\nUpdated data types after database alignment:")
print(df.dtypes)

# Check for any values that might be too long
print("\nMaximum string lengths:")
for col in string_columns:
    max_len = df[col].str.len().max()
    print(f"{col}: {max_len}")

# Display sample of numeric columns
print("\nSample of numeric columns:")
display(df[['Area', 'Init.PCE']].head())

## 4. Database Mapping Reference

Based on the database schema in `tables.sql`, our logbook data maps to the following tables:

### 1. scientist table
- Maps from: 'Owner', 'Producer' columns
- Fields:
  - scientist_id (UUID, generated)
  - name (from 'Owner' and 'Producer')

### 2. solar_cell_device table
- Maps from multiple columns
- Fields:
  - nomad_id (UUID, generated)
  - technology (from 'Cell type')
  - area (from 'Area')
  - initial_pce (from 'Init.PCE')
  - date_produced (can use 'Date installed')
  - encapsulation (from 'Encap.')
  - owner_id (link to scientist table)
  - producer_id (link to scientist table)

### 3. solar_cell_pixel table
- Maps from pixel-specific data
- Fields:
  - solar_cell_id (from generated device UUID)
  - pixel (from 'Pixel' column)
  - active_area (from 'Area')

### 4. mpp_tracking_channel table
- Maps from channel data
- Fields:
  - board (from 'Board')
  - channel (from 'Channel')

### 5. measurement_connection_event table
- Maps connection events
- Fields:
  - solar_cell_id (from generated device UUID)
  - pixel (from 'Pixel')
  - tracking_channel_board (from 'Board')
  - tracking_channel_channel (from 'Channel')
  - temperature_sensor_id (from 'Temp sensor')
  - connection_datetime (from 'Date installed')

This mapping shows we'll need to:
1. Generate UUIDs for new scientists and devices
2. Handle the relationships between tables
3. Convert dates to proper timestamp format
4. Validate data against database constraints

## 5. Prepare Data for Database Upload

We'll prepare the data for each table in the database schema, starting with the scientist table.

In [ ]:
import uuid

# Get unique scientists from both Owner and Producer columns
scientists = pd.concat([df['Owner'].dropna(), df['Producer'].dropna()]).unique()

# Create scientist dataframe
scientist_df = pd.DataFrame({
    'scientist_id': [uuid.uuid4() for _ in range(len(scientists))],
    'name': scientists
})

# Create a mapping dictionary for later use
scientist_id_map = dict(zip(scientist_df['name'], scientist_df['scientist_id']))

# Display the results
print(f"Found {len(scientist_df)} unique scientists")
print("\nScientist table preview:")
display(scientist_df)

print("\nValidating unique constraints:")
print(f"Duplicate names: {scientist_df['name'].duplicated().sum()}")
print(f"Duplicate UUIDs: {scientist_df['scientist_id'].duplicated().sum()}")

# Store the mapping for later use
print("\nScientist ID mapping (first few entries):")
for name, id in list(scientist_id_map.items())[:5]:
    print(f"{name}: {id}")

In [ ]:
# Clean up scientist data

# Remove NA values
scientist_df = scientist_df.dropna()

# Clean up names
# 1. Strip whitespace
# 2. Replace multiple spaces with single space
# 3. Remove any trailing commas or periods
scientist_df['name'] = scientist_df['name'].str.strip()
scientist_df['name'] = scientist_df['name'].str.replace(r'\s+', ' ', regex=True)
scientist_df['name'] = scientist_df['name'].str.replace(r'[,.]$', '', regex=True)

# Update the mapping dictionary
scientist_id_map = dict(zip(scientist_df['name'], scientist_df['scientist_id']))

# Display cleaned results
print(f"After cleaning, found {len(scientist_df)} scientists")
print("\nCleaned scientist table:")
display(scientist_df)

# Verify no duplicates or NA values remain
print("\nValidating cleaned data:")
print(f"Null values: {scientist_df['name'].isna().sum()}")
print(f"Duplicate names: {scientist_df['name'].duplicated().sum()}")
print(f"Duplicate UUIDs: {scientist_df['scientist_id'].duplicated().sum()}")

In [ ]:
# Additional cleaning: Remove <NA> strings and ensure no invalid values remain
print("Checking for '<NA>' values...")

# Check for '<NA>' strings
na_mask = scientist_df['name'].isin(['<NA>', 'NA', '<na>', 'na'])
if na_mask.any():
    print(f"Found {na_mask.sum()} '<NA>' values. Removing them...")
    scientist_df = scientist_df[~na_mask]

# Update the mapping dictionary again
scientist_id_map = dict(zip(scientist_df['name'], scientist_df['scientist_id']))

# Validate final results
print("\nFinal validation:")
print(f"Total scientists after removing <NA>: {len(scientist_df)}")
print(f"Any remaining NA values: {scientist_df['name'].isna().any()}")
print(f"Any remaining <NA> strings: {scientist_df['name'].isin(['<NA>', 'NA', '<na>', 'na']).any()}")

# Display final cleaned data
print("\nFinal cleaned scientist table:")
display(scientist_df)